In [1]:
import pandas as pd
import numpy as np

import unicodedata
import re
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
import requests
from fugashi import Tagger
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech

d:\PythonDevelopment\jap_lit\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
df = pd.read_csv('./data/books_final.csv', usecols=['id', 'char_count', 'text'])

df

,id,char_count,text
0,54333,80156.0,著者フレデリック・アークム専門化学者、ヨーロッパの主な芸術・科学アカデミーおよび学会の会員緒...
1,46340,76104.0,［＃改丁］序偉人の伝記というと、ナポレオンとかアレキサンドロスとか、グラッドストーンというよ...
2,46511,2975.0,人間が石にたよるやうになつて、もうよほど久しいことであるのに、まだ根気よくそれをやつてゐる。...
3,57975,2516.0,既に美育部を持つてゐる早稻田中學校が新に音樂會を興してその發會式をやらうとする其の日から、又...
4,57976,585.0,古い日記や手紙などを、みんな燒いてしまつたので、こまかに時日をいへないが、まだ若い中學教師で...
...,...,...,...
17365,49911,4761.0,問題にしない時にはわかり切ったことと思われているものが、さて問題にしてみると実にわからなくな...
17366,55622,12400.0,夢の話をするのはあまり気のきいたことではない。確か痴人夢を説くという言葉があったはずだ。そう...
17367,49876,4655.0,自分は現代の画家中に岸田君ほど明らかな「成長」を示している人を知らない。誇張でなく岸田君は一...
17368,49913,12075.0,一荒漠たる秋の野に立つ。星は月の御座を囲み月は清らかに地の花を輝らす。花は紅と咲き黄と匂い紫...


#### Preprocessing

In [20]:
df['char_count'].describe()

count     17370.000000
mean      13456.169315
std       34426.686869
min          14.000000
25%        1531.250000
50%        4314.000000
75%       11550.500000
max      823518.000000
Name: char_count, dtype: float64

In [12]:
len(df[df['char_count'] < 100])

406

In [47]:
def preprocess(txt):
    txt = re.sub(r'[^\w\s]', '', txt) # remove all punctuation
    txt = unicodedata.normalize('NFKC', txt) # standardizing japanese characters (全角 vs 半角)
    return txt

df['text'] = df['text'].apply(preprocess)


In [22]:
df['text'].head(5)

0    著者フレデリックアークム専門化学者ヨーロッパの主な芸術科学アカデミーおよび学会の会員緒言この...
1    改丁序偉人の伝記というとナポレオンとかアレキサンドロスとかグラッドストーンというようなのばか...
2    人間が石にたよるやうになつてもうよほど久しいことであるのにまだ根気よくそれをやつてゐる石にた...
3    既に美育部を持つてゐる早稻田中學校が新に音樂會を興してその發會式をやらうとする其の日から又病...
4    古い日記や手紙などをみんな燒いてしまつたのでこまかに時日をいへないがまだ若い中學教師であつた...
Name: text, dtype: object

In [48]:
def is_all_english(text):
    if bool(re.fullmatch(r'[A-Za-z\s]+', text)): # all english
        return True
    else:
        return False

df['all_eng'] = df['text'].apply(is_all_english)

In [33]:
df[df['all_eng']]

,id,char_count,text,all_eng
1279,55213,87.0,UrbinonowakdokosowaedakuminoichitotatreLeonard...,True
1280,55214,81.0,KonataniwaAbruzzinoyamakanataPontininonumaGeij...,True
8178,54811,100.0,IshikoronomichiIshikoronomichiKaattohigasenawo...,True
17218,888,96.0,HowkindhowfairshewashowgoodIcannottellyouIfIco...,True


In [49]:
df = df[~df['all_eng']].reset_index(drop=True)

df = df.drop(columns='all_eng')

In [65]:
df['text'] = df['text'].astype(str)

#### Embedding

In [66]:
text = df['text'].sample(1000)

In [71]:

embedding_model = SentenceTransformer("intfloat/multilingual-e5-small")

emb = embedding_model.encode(text.values, batch_size=4, show_progress_bar=True)


Batches: 100%|██████████| 250/250 [02:46<00:00,  1.51it/s]


In [72]:
emb.shape

(1000, 384)